# Excel Automation

# Getting Started with xlwings

## Using Excel as Data Viewer

In [2]:
# First, let's import the packages that we'll use in this chapter
import datetime as dt
import xlwings as xw
import pandas as pd
import numpy as np

In [3]:
# Let's create a DataFrame based on pseudorandom numbers and
# with enough rows that only the head and tail are shown
df = pd.DataFrame(data=np.random.randn(100, 5),
                  columns=[f"Trial {i}" for i in range(1, 6)])
df

,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5
0,1.092759,-0.906613,1.257995,-0.547838,-0.399337
1,-0.639740,-0.400355,1.742932,-0.655531,2.031878
2,0.159045,0.280051,0.180943,0.634040,1.168729
3,-0.668835,1.441644,-0.882267,0.961280,-1.766295
4,1.223944,-0.464663,-0.409358,0.854374,0.692102
...,...,...,...,...,...
95,0.761225,-0.520548,-0.140634,0.236864,-0.835785
96,-0.442922,-0.954625,0.741309,1.344989,-0.972110
97,-0.223218,-2.598526,0.181900,-0.384215,-0.012145
98,-1.207031,0.103273,-1.226342,0.838065,0.209997


In [4]:
# View the DataFrame in Excel
xw.view(df)

## The Excel Object Model

In [13]:
# Create a new empty workbook and print its name. This is the
# book we will use to run most of the code samples in this chapter.
book = xw.Book()
book.name

'Book1'

In [8]:
# Accessing the sheets collection
book.sheets

Sheets([<Sheet [Book1]Sheet1>])

In [15]:
# Get a sheet object by index or name. You will need to adjust
# "Sheet1" if your sheet is called differently.
sheet1 = book.sheets[0]
sheet1 = book.sheets["Sheet1"]

In [16]:
sheet1.range("A1")

<Range [Book1]Sheet1!$A$1>

In [18]:
# Most common tasks: write values...
sheet1.range("A1").value = [[1, 2],
                            [3, 4]]
sheet1.range("A4").value = "Hello!"

In [19]:
# ...and read values
sheet1.range("A1:B2").value

[[1.0, 2.0], [3.0, 4.0]]

In [20]:
sheet1.range("A4").value

'Hello!'

In [21]:
# Indexing
sheet1.range("A1:B2")[0, 0]

<Range [Book1]Sheet1!$A$1>

In [22]:
# Slicing
sheet1.range("A1:B2")[:, 1]

<Range [Book1]Sheet1!$B$1:$B$2>

In [23]:
# Single cell: A1 notation
sheet1["A1"]

<Range [Book1]Sheet1!$A$1>

In [ ]:
# Multiple cells: A1 notation
sheet1["A1:B2"]

In [ ]:
# Single cell: indexing
sheet1[0, 0]

In [ ]:
# Multiple cells: slicing
sheet1[:2, :2]

In [ ]:
# D10 via sheet indexing
sheet1[9, 3]

In [ ]:
# D10 via range object
sheet1.range((10, 4))

In [ ]:
# D10:F11 via sheet slicing
sheet1[9:11, 3:6]

In [ ]:
# D10:F11 via range object
sheet1.range((10, 4), (11, 6))

In [ ]:
sheet1["A1"].sheet.book.app

In [25]:
# Get one app object from the open workbook
# and create an additional invisible app instance
visible_app = sheet1.book.app
invisible_app = xw.App(visible=False)

In [26]:
# List the book names that are open in each instance
# by using a list comprehension
[book.name for book in visible_app.books]

['Book1']

In [27]:
[book.name for book in invisible_app.books]

['Book2']

In [28]:
# An app key represents the process ID (PID)
xw.apps.keys()

[6876, 14264]

In [29]:
# It can also be accessed via the pid attribute
xw.apps.active.pid

6876

In [30]:
# Work with the book in the invisible Excel instance
invisible_book = invisible_app.books[0]
invisible_book.sheets[0]["A1"].value = "Created by an invisible app."

In [ ]:
# Save the Excel workbook in the xl directory
invisible_book.save("xl/invisible.xlsx")

In [ ]:
# Quit the invisible Excel instance
invisible_app.quit()

## Running VBA Code

In [ ]:
vba_book = xw.Book("xl/vba.xlsm")

In [ ]:
# Instantiate a macro object with the VBA function
mysum = vba_book.macro("Module1.MySum")
# Call a VBA function
mysum(5, 4)

In [ ]:
# It works the same with a VBA Sub procedure
show_msgbox = vba_book.macro("Module1.ShowMsgBox")
show_msgbox("Hello xlwings!")

In [ ]:
# Close the book again (make sure to close the MessageBox first)
vba_book.close()

# Converters, Options and Collections

## Working with DataFrames

In [ ]:
data = [["Mark", 55, "Italy", 4.5, "Europe"],
        ["John", 33, "USA", 6.7, "America"]]
df = pd.DataFrame(data=data,
                  columns=["name", "age", "country",
                           "score", "continent"],
                  index=[1001, 1000])
df.index.name = "user_id"
df

In [ ]:
sheet1["A6"].value = df

In [ ]:
sheet1["B10"].options(header=False, index=False).value = df

In [ ]:
df2 = sheet1["A6"].expand().options(pd.DataFrame).value
df2

In [ ]:
# If you want the index to be an integer index,
# you can change its data type
df2.index = df2.index.astype(int)
df2

In [ ]:
# By setting index=False, it will put all the values from Excel into
# the data part of the DataFrame and will use the default index
sheet1["A6"].expand().options(pd.DataFrame, index=False).value

## Converters and Options

In [ ]:
# Horizontal range (one-dimensional)
sheet1["A1:B1"].value

In [ ]:
# Vertical range (one-dimensional)
sheet1["A1:A2"].value

In [ ]:
# Horizontal range (two-dimensional)
sheet1["A1:B1"].options(ndim=2).value

In [ ]:
# Vertical range (two-dimensional)
sheet1["A1:A2"].options(ndim=2).value

In [ ]:
# Using the NumPy array converter behaves the same:
# vertical range leads to a one-dimensional array
sheet1["A1:A2"].options(np.array).value

In [ ]:
# Preserving the column orientation
sheet1["A1:A2"].options(np.array, ndim=2).value

In [ ]:
# If you need to write out a list vertically,
# the "transpose" option comes in handy
sheet1["D1"].options(transpose=True).value = [100, 200]

In [ ]:
# Write out some sample data
sheet1["A13"].value = [dt.datetime(2020, 1, 1), None, 1.0]

In [ ]:
# Read it back using the default options
sheet1["A13:C13"].value

In [ ]:
# Read it back using non-default options
sheet1["A13:C13"].options(empty="NA",
                          dates=dt.date,
                          numbers=int).value

## Charts, Pictures and Defined Names

In [ ]:
sheet1["A15"].value = [[None, "North", "South"],
                       ["Last Year", 2, 5],
                       ["This Year", 3, 6]]

In [ ]:
chart = sheet1.charts.add(top=sheet1["A19"].top,
                          left=sheet1["A19"].left)
chart.chart_type = "column_clustered"
chart.set_source_data(sheet1["A15"].expand())

In [ ]:
# Read in the chart data as DataFrame
df = sheet1["A15"].expand().options(pd.DataFrame).value
df

In [ ]:
# Enable Matplotlib by using the notebook magic command
# and switch to the "seaborn" style
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [ ]:
# The pandas plot method returns an "axis" object from
# where you can get the figure. "T" transposes the
# DataFrame to bring the plot into the desired orientation
ax = df.T.plot.bar()
fig = ax.get_figure()

In [ ]:
# Send the plot to Excel
plot = sheet1.pictures.add(fig, name="SalesPlot",
                           top=sheet1["H19"].top,
                           left=sheet1["H19"].left)
# Let's scale the plot to 70%
plot.width, plot.height = plot.width * 0.7, plot.height * 0.7

In [ ]:
ax = (df + 1).T.plot.bar()
plot = plot.update(ax.get_figure())

In [ ]:
# The book scope is the default scope
sheet1["A1:B2"].name = "matrix1"

In [ ]:
# For the sheet scope, prepend the sheet name with
# an exclamation point
sheet1["B10:E11"].name = "Sheet1!matrix2"

In [ ]:
# Now you can access the range by name
sheet1["matrix1"]

In [ ]:
# If you access the names collection via the "sheet1" object,
# it contains only names with that sheet's scope
sheet1.names

In [ ]:
# If you access the names collection via the "book" object,
# it contains all names, including book and sheet scope
book.names

In [ ]:
# Names have various methods and attributes.
# You can, for example, get the respective range object.
book.names["matrix1"].refers_to_range

In [ ]:
# If you want to assign a name to a constant
# or a formula, use the "add" method.
# You may need to replace the decimal point with a comma
# if your are using an international version of Excel.
book.names.add("EURUSD", "=1.1151")

# Advanced Topics

## Performance

In [ ]:
# Add a new sheet and write 150 values
# to it to have something to work with
sheet2 = book.sheets.add()
sheet2["A1"].value = np.arange(150).reshape(30, 5)

In [ ]:
%%time
# This makes 150 cross-application calls
for cell in sheet2["A1:E30"]:
    cell.value += 1

In [ ]:
%%time
# This makes just two cross-application calls
values = sheet2["A1:E30"].options(np.array).value
sheet2["A1:E30"].value = values + 1

In [ ]:
# With raw values, you must provide the full
# target range, sheet["A35"] doesn't work anymore
sheet1["A35:B36"].options("raw").value = [[1, 2], [3, 4]]